# 智慧拼图游戏

目标：通过 A* 算法将随机状态的拼图恢复到目标状态。

![title](other_data/01.jpg)


## 创建一个随机布局

In [ ]:
import random

digits = [0,1,2,3,4,5,6,7,8]

#将数字打乱
random.shuffle(digits)

input_layout = ''
for numb in digits:
    input_layout+=str(numb)
    
print(input_layout)

## 九宫格显示布局

In [ ]:
def show_arrange(str_digits):
    row = []
    for i in range(len(str_digits)):
        row.append(int(str_digits[i]))
        if (i+1)%3==0:
            print(row)
            row = []

show_arrange(input_layout)

## 判断该布局是否能够恢复到目标布局
先进行判断input_layout和target_layout逆序值是否同是奇数或偶数,如果不是就没有解
* 逆序值: 记录当前状态中除0以外每一个值大于后面所有值的个数
    1. 如果当前状态为 [3,1,2,0,4,5,6,7,8]；
    2. 满足当前状态中的值大于后面的值的情况有 3>1,3>2，有2个逆序值，也就有偶数个逆序值；
    3. 如果目标状态为[0,1,2,3,4,5,6,7,8],那么目标状态的逆序值就为0，0也为偶数，所以状态[3,1,2,0,4,5,6,7,8]可以恢复到[0,1,2,3,4,5,6,7,8]。

In [ ]:
# 设置状态
# input_layout = '312045678'

# 目标状态
target_layout = '012345678'

def able_change(layout):
    num_bigger  = 0
    for i in range(1,9):
        bigger=0
        for j in range(0,i):
            # 不记录数字 0 的逆序值
            if layout[j]>layout[i] and layout[i]!='0':
                bigger=bigger+1
        num_bigger=num_bigger+bigger
    return num_bigger

input_result = able_change(input_layout)
target_result = able_change(target_layout)

if (input_result%2)==(target_result%2):
    able_change = True
    show_arrange(input_layout)
    print('该布局可以变为目标状态')
else:
    able_change=False
    print('该布局不能变为目标状态') 

## 曼哈顿距离
计算当前布局恢复到目标位置的步数 hn 

In [ ]:
# 将字符串转换为矩阵形式
def str2list(curLayout):
    matrix = []
    row = []
    for i in range(len(curLayout)):
        row.append(int(curLayout[i]))
        if (i+1) % 3==0:
            matrix.append(row)
            row = []
    return matrix

# 计算hn
def cal_hn(curLayout, target_layout):
    hn = 0
    curLayout = str2list(curLayout)
    target_layout = str2list(target_layout)
    # 当前状态的行列
    for i in range(len(curLayout)):
        for j in range(len(curLayout[i])):
            # 目标状态的行列
            for m in range(len(target_layout)):
                for n in range(len(target_layout[m])):
                    # 如果当前状态的数字等于目标状态的数字
                    if curLayout[i][j]==target_layout[m][n]:
                        # 当前目标状态的数字的坐标为[m,n]
                        pos = [m,n]
                        # 当前hn等于目标状态的每一个数字坐标减去当前状态的每一个数字坐标
                        hn += abs(pos[0] - i) + abs(pos[1] - j)
    return hn

## A*算法
移动当前状态中数字0到相邻位置，每次移动一步
参数说明：
1. curLayout：当前布局；
2. swp_index：数字0可以移动的位置索引，当前我们移动数字0时，每次只能移动一步，因此每个位置都有对应移动的位置索引；
    * 比如当前状态为 [3,1,2,0,4,5,6,7,8]，数字 0 的索引为 3 ，那么数字 0 就可以移动到 [0, 4, 6]三个位置，那么我们就可以用一个字典将每个位置的可移动的位置索引记录下来。
3. zero_index：当前布局中数字 0 的索引值；
4. layouts_gn：当前的gn值，该程序循环的步数；
5. target_layout：目标布局。

In [ ]:
#每个位置可交换的位置集合
able_swp = {0:[1, 3], 1:[0, 2, 4], 2:[1, 5],
                 3:[0,4,6], 4:[1,3,5,7], 5:[2,4,8],
                 6:[3,7],  7:[4,6,8], 8:[5,7]}

def swap_chr(curLayout, swp_index, zero_index, layouts_gn, target_layout):
    # 如果移动索引值 大于数字0索引值，交换两个的索引值
    if swp_index > zero_index:
        swp_index, zero_index = zero_index, swp_index
    #得到交换后的数组
    # 例子 ： 如果当前状态为 [3,1,2,0,4,5,6,7,8]，数字 0 可以移动到 [0, 4, 6]三个位置，如果当前状态中数字0往右移动，移动到索引 4 ,4(swp_index) > 3(zero_index),
    # 交换两个索引值 swp_index = 3 ，zero_index = 4
    # 1. curLayout[:swp_index] = curLayout[:3] = [3,1,2]
    # 2. curLayout[zero_index] = curLayout[4] = [4]
    # 3. curLayout[swp_index+1:zero_index] = curLayout[4:4] = []
    # 4. curLayout[swp_index] = curLayout[3] = [0]
    # 5. curLayout[zero_index+1:] = curLayout[5:] = [5,6,7,8]
    # layout = #1 + #2 + #3 + #4 + #5 = [3,1,2,4,0,5,6,7,8]
    layout = curLayout[:swp_index] + curLayout[zero_index] + curLayout[swp_index+1:zero_index] + curLayout[swp_index] + curLayout[zero_index+1:]
    # A*算法算出fn
    fn = cal_hn(layout, target_layout)+layouts_gn
    return layout, fn

## 记录交换后的信息值
1. g_dict_layouts：根据fn值记录的所有变化的布局，上一次状态与新状态的字典
2. layouts_gn：记录布局状态与gn值的字典
3. layouts_fn：记录布局状态与fn值的字典

In [ ]:
g_dict_layouts = {}
layouts_gn = {}
layouts_fn = {}

g_dict_layouts[input_layout] = 0
for keys,values in g_dict_layouts.items():
    print('当前布局为\n')
    show_arrange(keys)

# 初始gn设置为1
layouts_gn[input_layout] = 1
for keys,values in layouts_gn.items():
    print('\ngn值为：{}'.format(values))

# fn = hn + gn
layouts_fn[input_layout] = cal_hn(input_layout, target_layout) + layouts_gn[input_layout]

for keys,values in layouts_fn.items():
    print('\nfn值为：{}'.format(values))

## A*算法自动恢复布局

In [ ]:

if able_change==True:
    while True:
        # 找出字典layouts_fn中fn最小的布局
        curLayout = min(layouts_fn, key=layouts_fn.get)
        
        # 因为每次要在字典里面找出fn最小的布局，因此需要删除当前fn最小的布局
        del layouts_fn[curLayout]
        
        #判断当前状态是否为目标状态
        if curLayout == target_layout:
            break

        # 寻找 0 的位置。
        zero_index = curLayout.index("0")

        #当前可进行交换的位置集合
        swaps = able_swp[zero_index]

        for swp_index in swaps:
            # 给“0”交换位置,返回交换后的布局与fn值
            newLayout, fn = swap_chr(curLayout, swp_index, zero_index, layouts_gn[curLayout]+1, target_layout)
            
            #判断新布局是否在字典g_dict_layouts中
            if g_dict_layouts.get(newLayout) == None:

                # 保存字典{布局：gn}， gn：在当前curLayout布局状态下+1
                layouts_gn[newLayout] = layouts_gn[curLayout]+1
                
                # 保存字典{布局：fn}
                layouts_fn[newLayout] = fn

                # 记录布局的更新 {newLayout：curLayout}
                g_dict_layouts[newLayout] = curLayout 
else:
    print('该状态布局不能恢复到目标状态')

## 打印布局恢复过程

In [ ]:
try:
    lst_steps = []
    lst_steps.append(curLayout)
    while g_dict_layouts[curLayout] != 0:#存入路径
        curLayout = g_dict_layouts[curLayout]
        lst_steps.append(curLayout)
    lst_steps.reverse()

    for nIndex in range(len(lst_steps)):
        print("第%d步:" %(nIndex + 1))
        show_arrange(lst_steps[nIndex])
except:
    print('该状态布局不能恢复到目标状态')